# The feedforward neural network model for learning PageRank

In [1]:
import time
from mygraph import MyGraph
from helpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("fb_media_pages_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([4.26055693e-05])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 24000
target_pagerank[0]

array([1.02253366])

In [5]:
target_pagerank.shape

(27917, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("fb_media_pages_NDFC_matrix_r1_3_sta1_max70_rad5.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(27917, 6, 23)

In [8]:
matrix_1[111]

array([[0.      , 0.      , 0.      , 2.      , 2.      , 0.      ,
        1.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [0.6     , 0.      , 0.2     , 2.8     , 1.6     , 0.      ,
        1.      , 0.4     , 0.4     , 0.      , 0.2     , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.2     , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      ],
       [1.      , 0.583333, 0.333333, 1.75    , 2.916667, 3.583333,
        4.166667, 4.166667, 4.666667, 2.5     , 1.5     , 0.916667,
        0.75    , 0.333333, 0.416667, 0.25    , 0.583333, 0.083333,
        0.      , 0.166667, 0.      , 0.083333, 0.      ],
       [0.252941, 0.514706, 0.594118, 1.326471, 1.461765, 2.467647,
        2.626471, 3.364706, 3.952941, 3.4     , 2.897059, 1.888235,
        1.714706, 1.25    , 1.052941, 0.479412, 1.082353, 0.208824,
       

In [9]:
matrix_1 = matrix_1.reshape(27917,-1)
matrix_1.shape

(27917, 138)

# Converting the data into pytorch tenors

In [10]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([27917, 138]), torch.Size([27917, 1]))

In [11]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[1.0225],
        [0.9398],
        [0.8766],
        ...,
        [0.4565],
        [0.9406],
        [0.9156]], dtype=torch.float64, requires_grad=True)

In [12]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 7917
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([20000, 138]),
 torch.Size([7917, 138]),
 torch.Size([20000, 1]),
 torch.Size([7917, 1]))

## A function for dividing train data into batches

In [13]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [14]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [15]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=138, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [16]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

55200
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
549993


In [17]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [18]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [19]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.71360893205,    TEST LOSS:   0.69244498014
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.14950893819,    TEST LOSS:   0.55174398422
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.09811637551,    TEST LOSS:   0.48283165693
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.06834223121,    TEST LOSS:   0.43720608950
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.05121678486,    TEST LOSS:   0.40616497397
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.03991750628,    TEST LOSS:   0.38239413500
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.03141590208,    TEST LOSS:   0.36389908195
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.02575652674,    TEST LOSS:   0.35057145357
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [20]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,138)
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%50 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 0.2531810150032319,    prediction: 0.29693353176116943
index 0:       inaccuracy: 17.281%
-----------------------------------------------------------------
target 0.31066267415386484,    prediction: 0.3176133334636688
index 50:       inaccuracy: 2.237%
-----------------------------------------------------------------
target 0.5609903243209725,    prediction: 0.5689537525177002
index 100:       inaccuracy: 1.42%
-----------------------------------------------------------------
target 0.22848819632453027,    prediction: 0.31601962447166443
index 150:       inaccuracy: 38.309%
-----------------------------------------------------------------
target 0.4082360811373197,    prediction: 0.549639880657196
index 200:       inaccuracy: 34.638%
-----------------------------------------------------------------
target 1.6270324947333759,    prediction: 1.4713375568389893
index 250:       inaccuracy: 9.569%
-----------------------------------------------------------------
target 1.09673273738

target 0.617423587783989,    prediction: 0.5906199812889099
index 2550:       inaccuracy: 4.341%
-----------------------------------------------------------------
target 0.4946835411208935,    prediction: 0.5334507822990417
index 2600:       inaccuracy: 7.837%
-----------------------------------------------------------------
target 15.050027973057151,    prediction: 5.86257266998291
index 2650:       inaccuracy: 61.046%
-----------------------------------------------------------------
target 0.44610127781607867,    prediction: 0.505761981010437
index 2700:       inaccuracy: 13.374%
-----------------------------------------------------------------
target 0.4358173330864417,    prediction: 0.3999174237251282
index 2750:       inaccuracy: 8.237%
-----------------------------------------------------------------
target 0.7963927383852735,    prediction: 0.7361339330673218
index 2800:       inaccuracy: 7.566%
-----------------------------------------------------------------
target 0.19184461

target 0.7958680293702163,    prediction: 0.7258882522583008
index 5100:       inaccuracy: 8.793%
-----------------------------------------------------------------
target 0.2960613489965423,    prediction: 0.31897324323654175
index 5150:       inaccuracy: 7.739%
-----------------------------------------------------------------
target 2.0661064872272528,    prediction: 2.0145788192749023
index 5200:       inaccuracy: 2.494%
-----------------------------------------------------------------
target 0.9077696090735978,    prediction: 0.9143128991127014
index 5250:       inaccuracy: 0.721%
-----------------------------------------------------------------
target 0.7613515328800387,    prediction: 0.8366528749465942
index 5300:       inaccuracy: 9.89%
-----------------------------------------------------------------
target 0.5556080680662466,    prediction: 0.5270494818687439
index 5350:       inaccuracy: 5.14%
-----------------------------------------------------------------
target 0.69083949

target 0.46673936348340656,    prediction: 0.4215255379676819
index 7700:       inaccuracy: 9.687%
-----------------------------------------------------------------
target 2.0719310510757762,    prediction: 2.088580369949341
index 7750:       inaccuracy: 0.804%
-----------------------------------------------------------------
target 0.4171119666518242,    prediction: 0.49171000719070435
index 7800:       inaccuracy: 17.884%
-----------------------------------------------------------------
target 1.2773336073867856,    prediction: 1.1915578842163086
index 7850:       inaccuracy: 6.715%
-----------------------------------------------------------------
target 0.42287251669274883,    prediction: 0.42797788977622986
index 7900:       inaccuracy: 1.207%
-----------------------------------------------------------------


In [21]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  12.438


In [22]:
torch.save(model, "trained_FFNN_model_fb_media_pagerank_001.pt")